# Berkeley Course Crawler

## Functions (You can skip this section, but remember to run the cell)
How to run? Just move your cursor to the cell of code and click on "Run" button.

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  4 23:32:27 2020

@author: You, Bo-Xiang
"""

def course_extract(url):
    
    import requests
    from datetime import datetime, timedelta
    import json
    import pandas as pd
    from bs4 import BeautifulSoup
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    query = soup.find("div", class_="handlebarData theme_is_whitehot").attrs
    js_str = query["data-json"]
    js_dict = json.loads(js_str)
    course_num = js_dict["displayName"]
    serial_num = js_dict["id"]
    name = js_dict["course"]["title"]
    level = js_dict["course"]["academicCareer"]["description"]
    lecture = js_dict["component"]["description"]
    
    try:
        mode = js_dict["attributes"]["WEB"][0]["value"]["formalDescription"]
    except KeyError:
        mode = "Pending Reviews"
    
    instructor = js_dict["meetings"][0]["assignedInstructors"]
    instructors = ', '.join([str(i["instructor"]["names"][1]["formattedName"]) for i in instructor])
    
    try:
        units = js_dict["course"]["credit"]["value"]["fixed"]["units"]
    except KeyError:
        units = ' to '.join([str(js_dict["course"]["credit"]["value"]["range"]["minUnits"]), str(js_dict["course"]["credit"]["value"]["range"]["maxUnits"])])
    
    # date-time process
    fake_date = []
    if js_dict["meetings"][0]["meetsMonday"]:
        fake_date.append(datetime(2020,7,6).date())
    if js_dict["meetings"][0]["meetsTuesday"]:
        fake_date.append(datetime(2020,7,7).date())
    if js_dict["meetings"][0]["meetsWednesday"]:
        fake_date.append(datetime(2020,7,8).date())
    if js_dict["meetings"][0]["meetsThursday"]:
        fake_date.append(datetime(2020,7,9).date())
    if js_dict["meetings"][0]["meetsFriday"]:
        fake_date.append(datetime(2020,7,10).date())
    fake_starttime = datetime.strptime(js_dict["meetings"][0]["startTime"], '%H:%M:%S').time()
    fake_endtime = datetime.strptime(js_dict["meetings"][0]["endTime"], '%H:%M:%S').time()
    fake_start_dt = [datetime.combine(i, fake_starttime) for i in fake_date]
    fake_end_dt = [datetime.combine(i, fake_endtime) for i in fake_date]
    
    sf_start_dt = " / ".join([str(i.strftime("%a %H:%M")) for i in fake_start_dt])
    sf_end_dt = " / ".join([str(i.strftime("%a %H:%M")) for i in fake_end_dt])
    
    tw_start_dt = " / ".join([str((i + timedelta(hours=9)).strftime("%a %H:%M")) for i in fake_start_dt])
    tw_end_dt = " / ".join([str((i + timedelta(hours=9)).strftime("%a %H:%M")) for i in fake_end_dt])
    
    description = js_dict["course"]["description"]
    final = js_dict["course"]["finalExam"]["description"]
    
    information = pd.DataFrame({
            "Course_No." : course_num,
            "Serial_NO." : serial_num,
            "Course_Name" : name,
            "Level" : level,
            "Type" : lecture,
            "Mode" : mode,
            "Instructor(s)" : instructors,
            "Units" : units,
            "Start_Time(SF_time)" : sf_start_dt,
            "End_Time(SF_time)" : sf_end_dt,
            "Start_Time(TW_time)" : tw_start_dt,
            "End_Time(TW_time)" : tw_end_dt,
            "Final_Examination" : final,
            "Description" : description}, index=[0])
    
    return information

def multiple_extract(url_list):
    import pandas as pd
    information =  pd.DataFrame({
            "Course_No." : None,
            "Serial_NO." : None,
            "Course_Name" : None,
            "Level" : None,
            "Type" : None,
            "Mode" : None,
            "Instructor(s)" : None,
            "Units" : None,
            "Start_Time(SF_time)" : None,
            "End_Time(SF_time)" : None,
            "Start_Time(TW_time)" : None,
            "End_Time(TW_time)" : None,
            "Final_Examination" : None,
            "Description" : None}, index=[0])
    for i in url_list:
        info = course_extract(i)
        information = information.append(info, ignore_index=True)
    return information.iloc[1:]


### Setting style

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

## Demonstration

In [3]:
# Example for one Course: 
result = course_extract("https://classes.berkeley.edu/content/2020-fall-indeng-290-004-lec-004") # put the link between ""
result

Course_No.  Serial_NO.  \
0  2020 Fall INDENG 290 004 LEC 004       32956   

                                         Course_Name     Level     Type  \
0  Special Topics in Industrial Engineering and O...  Graduate  Lecture   

                       Mode Instructor(s)   Units    Start_Time(SF_time)  \
0  Asynchronous Instruction   Barna  Saha  2 to 3  Tue 14:00 / Thu 14:00   

       End_Time(SF_time)    Start_Time(TW_time)      End_Time(TW_time)  \
0  Tue 15:29 / Thu 15:29  Tue 23:00 / Thu 23:00  Wed 00:29 / Fri 00:29   

                                   Final_Examination  \
0  Written final exam conducted during the schedu...   

                                         Description  
0  Lectures and appropriate assignments on fundam...

In [4]:
# copy the result and paste it on your excel/Google Sheet/Database
result.to_clipboard(excel=True,sep='\t')

In [5]:
# Example for multiple Courses:
# 1. Put all your desired course link into the list, separate them by ","
url_list = ["https://classes.berkeley.edu/content/2020-fall-indeng-290-004-lec-004", "https://classes.berkeley.edu/content/2020-fall-civeng-199-001-ind-001", "https://classes.berkeley.edu/content/2020-fall-indeng-290-004-lec-004"]
# 2. Extract them as table
multiple_results = multiple_extract(url_list)
multiple_results

Course_No. Serial_NO.  \
1  2020 Fall INDENG 290 004 LEC 004      32956   
2  2020 Fall CIVENG 199 001 IND 001      16638   
3  2020 Fall INDENG 290 004 LEC 004      32956   

                                         Course_Name          Level  \
1  Special Topics in Industrial Engineering and O...       Graduate   
2                       Supervised Independent Study  Undergraduate   
3  Special Topics in Industrial Engineering and O...       Graduate   

                Type                      Mode        Instructor(s)   Units  \
1            Lecture  Asynchronous Instruction          Barna  Saha  2 to 3   
2  Independent Study           Pending Reviews  Norman A Abrahamson  1 to 4   
3            Lecture  Asynchronous Instruction          Barna  Saha  2 to 3   

     Start_Time(SF_time)      End_Time(SF_time)    Start_Time(TW_time)  \
1  Tue 14:00 / Thu 14:00  Tue 15:29 / Thu 15:29  Tue 23:00 / Thu 23:00   
2                                                                        
3  Tue 14:00 / Thu 14:00  Tue 15:29 / Thu 15:29  Tue 23:00 / Thu 23:00   

       End_Time(TW_time)                                  Final_Examination  \
1  Wed 00:29 / Fri 00:29  Written final exam conducted during the schedu...   
2                                                             No final exam   
3  Wed 00:29 / Fri 00:29  Written final exam conducted during the schedu...   

                                         Description  
1  Lectures and appropriate assignments on fundam...  
2                     Supervised independent study.   
3  Lectures and appropriate assignments on fundam...

In [6]:
# 3. use this action to copy the result and paste it on your excel/Google Sheet/Database
multiple_results.to_clipboard(excel=True,sep='\t')

## Search for the courses that interest you

In [ ]:
# NOW, Try it on your own for one link!
result = course_extract("") # insert link
result.to_clipboard(excel=True,sep='\t') # copy the result, so you can paste it elsewhere
#check the result
result

In [ ]:
# NOW, Try it on your own for multiple link!
url_list = ["", "", ""] # insert links, you can add as more links as you want
multiple_results = multiple_extract(url_list)
multiple_results.to_clipboard(excel=True,sep='\t') # copy the result, so you can paste it elsewhere
#check the result
multiple_results